In [1]:
import json 
import networkx as nx
  

from conjecture.consolidated_functions import *
from conjecture.all_valid_orderings import *
from conjecture.optimization_functions import *
from iterative_heuristic.modified_etf import Mod_ETF
from iterative_heuristic.consolidated import *
from iterative_heuristic.approximate_speeds import *
from makespan_energy.construct_graph_util import *
from makespan_energy.visualization_util import *
from trace_data.graph_list_from_trace import *
import networkx as nx
import numpy as np
import math
import plotly.express as px
import pandas as pd 

In [2]:
# Construct DAGs

file_list = ['1000genome-chameleon-2ch-100k-001.json',
'1000genome-chameleon-2ch-250k-001.json',
'1000genome-chameleon-4ch-100k-001.json',
'1000genome-chameleon-4ch-250k-001.json',
'1000genome-chameleon-6ch-100k-001.json',
'1000genome-chameleon-6ch-250k-001.json',
'1000genome-chameleon-8ch-100k-001.json',
'1000genome-chameleon-8ch-250k-001.json',
'1000genome-chameleon-10ch-100k-001.json',
'1000genome-chameleon-10ch-250k-001.json',
'1000genome-chameleon-12ch-100k-001.json',
'1000genome-chameleon-12ch-250k-001.json',
'1000genome-chameleon-14ch-100k-001.json',
'1000genome-chameleon-14ch-250k-001.json',
'1000genome-chameleon-16ch-100k-001.json',
'1000genome-chameleon-16ch-250k-001.json',
'1000genome-chameleon-18ch-100k-001.json',
'1000genome-chameleon-18ch-250k-001.json',
'1000genome-chameleon-20ch-100k-001.json',
'1000genome-chameleon-20ch-250k-001.json',
'1000genome-chameleon-22ch-100k-001.json',
'1000genome-chameleon-22ch-250k-001.json']
    
G_list = []

for file in file_list:
    
    # Opening JSON file 
    f = open('pegasus-traces-master/1000genome/chameleon-cloud/' + file) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 

    taskname_to_id = {}
    id_count = 0

    G = nx.DiGraph()

    for task in data['workflow']['jobs']:

        child_id = id_count
        id_count += 1
        taskname_to_id[task['name']] = child_id

        if not G.has_node(child_id):
                G.add_node(child_id)

        for parent in task['parents']:
            if parent in taskname_to_id:

                parent_id = taskname_to_id[parent]

            else:

                parent_id = id_count
                id_count += 1
                taskname_to_id[parent] = parent_id


            if not G.has_edge(parent_id, child_id):
                    G.add_edge(parent_id, child_id)
    print(len(G.nodes))

    
   
    G_list.append(G)
    
    # Closing file 
    f.close() 

52
82
104
164
156
246
208
328
260
410
312
492
364
574
416
656
468
738
520
820
572
902


In [3]:
G_list[0]

In [ ]:
mrt = True
tie_breaking_rule = 2
plot = False

x = []
y = []
lbs= []
ubs = []
 
num_machines = 3
naives = []
heur = []
for G in G_list:
 
    ratio = []
    num_tasks = len(G.nodes)
    
    w = [1 for _ in range(num_tasks)]
    s = [1 for _ in range(num_tasks)]

    naive, true, order =iterative_and_naive_heuristic_no_ratio(num_machines, w, G, verbose=False)
    _, _, obj_opt, _ = get_objective_single_ordering(True, G, w, order, plot=plot, compare=False)
    d = get_optimal_schedule(mrt, G, num_machines, w, plot, verbose=False)
    # if solver was successful
    # and d["objective_task_scaling"] != 10000000
    #if obj_opt != 10000000 and d["objective_task_scaling"] != 10000000:
        
    x.append(num_tasks)
    heur.append(true)
    naives.append(naive)
    y.append(obj_opt)
    lbs.append(d["objective_task_scaling"])
    ub = obj_ub_precise = 2 * sum(psize_to_speed(ub_lst_graph(G, order)))
    ubs.append(ub)
 

[17, 17, 16, 16, 16, 15, 15, 15, 14, 14, 14, 18, 13, 13, 13, 12, 12, 12, 11, 11, 11, 10, 10, 17, 10, 9, 9, 9, 8, 8, 8, 7, 7, 7, 6, 6, 6, 5, 5, 5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1]
[17.333333333333336, 16.333333333333336, 16.333333333333336, 16.333333333333336, 15.333333333333334, 15.333333333333334, 15.333333333333334, 14.333333333333334, 14.333333333333334, 14.333333333333334, 13.5, 17.333333333333336, 13.5, 13, 12.5, 12.5, 12, 11.5, 11.5, 11, 10.5, 10.5, 10, 17.333333333333336, 10, 9, 9, 9, 8, 8, 8, 7, 7, 7, 6, 6, 6, 5, 5, 5, 4, 4, 4, 3, 3, 3, 2, 2, 2, 1, 1, 1]
-
apm 172.8.132.240_gk_model1 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :           57
   Constants    :            0
   Variables    :          991
   Intermediates:            0
   Connec

apm 172.8.132.240_gk_model3 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :           87
   Constants    :            0
   Variables    :         1801
   Intermediates:            0
   Connections  :          743
   Equations    :         1386
   Residuals    :         1386
 
 Number of state variables:           1801
 Number of total equations: -         1472
 Number of slack variables: -          973
 ---------------------------------------
 Degrees of freedom       :           -644
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

**************************************************************************

apm 172.8.132.240_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          109
   Constants    :            0
   Variables    :         1971
   Intermediates:            0
   Connections  :          941
   Equations    :         1446
   Residuals    :         1446
 
 Number of state variables:           1971
 Number of total equations: -         1554
 Number of slack variables: -          923
 ---------------------------------------
 Degrees of freedom       :           -506
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

**************************************************************************

apm 172.8.132.240_gk_model7 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          169
   Constants    :            0
   Variables    :         3591
   Intermediates:            0
   Connections  :         1481
   Equations    :         2766
   Residuals    :         2766
 
 Number of state variables:           3591
 Number of total equations: -         2934
 Number of slack variables: -         1943
 ---------------------------------------
 Degrees of freedom       :          -1286
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

**************************************************************************

apm 172.8.132.240_gk_model9 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          161
   Constants    :            0
   Variables    :         2951
   Intermediates:            0
   Connections  :         1409
   Equations    :         2166
   Residuals    :         2166
 
 Number of state variables:           2951
 Number of total equations: -         2326
 Number of slack variables: -         1383
 ---------------------------------------
 Degrees of freedom       :           -758
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

**************************************************************************

apm 172.8.132.240_gk_model11 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          251
   Constants    :            0
   Variables    :         5381
   Intermediates:            0
   Connections  :         2219
   Equations    :         4146
   Residuals    :         4146
 
 Number of state variables:           5381
 Number of total equations: -         4396
 Number of slack variables: -         2913
 ---------------------------------------
 Degrees of freedom       :          -1928
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

*************************************************************************

apm 172.8.132.240_gk_model13 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          213
   Constants    :            0
   Variables    :         3931
   Intermediates:            0
   Connections  :         1877
   Equations    :         2886
   Residuals    :         2886
 
 Number of state variables:           3931
 Number of total equations: -         3098
 Number of slack variables: -         1843
 ---------------------------------------
 Degrees of freedom       :          -1010
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

*************************************************************************

-
apm 172.8.132.240_gk_model15 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :          333
   Constants    :            0
   Variables    :         7171
   Intermediates:            0
   Connections  :         2957
   Equations    :         5526
   Residuals    :         5526
 
 Number of state variables:           7171
 Number of total equations: -         5858
 Number of slack variables: -         3883
 ---------------------------------------
 Degrees of freedom       :          -2570
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

***********************************************************************

[83.66666666666666, 83.66666666666666, 82.66666666666666, 82.66666666666666, 82.66666666666666, 81.66666666666666, 81.66666666666666, 81.66666666666666, 80.66666666666666, 80.66666666666666, 79.66666666666666, 86.66666666666666, 80.66666666666666, 79.66666666666666, 79.66666666666666, 78.66666666666666, 78.66666666666666, 78.66666666666666, 77.66666666666666, 77.66666666666666, 77.66666666666666, 76.66666666666666, 75.66666666666666, 86.66666666666666, 76.66666666666666, 76.66666666666666, 75.66666666666666, 75.66666666666666, 74.66666666666666, 74.66666666666666, 74.66666666666666, 73.66666666666666, 73.66666666666666, 73.66666666666666, 72.66666666666666, 86.66666666666666, 72.66666666666666, 72.66666666666666, 71.66666666666666, 71.66666666666666, 71.66666666666666, 70.66666666666666, 70.66666666666666, 70.66666666666666, 69.66666666666666, 69.66666666666666, 68.66666666666666, 85.66666666666666, 69.66666666666666, 68.66666666666666, 68.66666666666666, 67.66666666666666, 67.66666666

In [ ]:
print("x is \n", x)
print("y is \n", y)
print("lbs are \n",lbs)
print("ubs are \n", ubs)
print("naives are \n",naives)
print("heurs are \m",heur)

In [ ]:
print(len(y))
print(len(x))

In [ ]:
# naives = [572.8281617409056, 1331.3128540244006, 2051.988186148974, 4908.957570715966, 4427.097315671543, 10722.551392522024, 7706.784325022744, 18780.723094156718, 11887.994968277473, 29080.41842969494, 16967.104097800042, 41618.012251501, 22950.791107151243, 56400.18395313568, 29836.001750405976, 73423.87928867387, 37619.11087992854, 92685.47311048009, 46306.79788927975, 114191.64481211497, 55896.00853253448, 137939.34014765336]
# heurs = [351.14748449053826, 698.6688417479015, 1181.833119651485, 2392.931559303018, 2515.1051913091133, 5128.200522698374, 4320.182955770601, 8876.07384841392, 6602.766119158128, 13562.092685406415, 9411.33279943706, 19356.644767630198, 12670.852536835615, 26103.094518568214, 16404.64052815484, 33831.316526414616, 20688.501817054886, 42685.15172746671, 25668.437288627654, 52905.78716201741, 30587.45634664135, 63200.60308232752]
# y = [300.05921, 586.18851, 10000000, 1637.77108, 1520.32995, 10000000, 2333.07674, 10000000, 3254.04157, 6424.66253, 4271.75539, 10000000, 5377.87606, 10000000, 6565.71298, 12977.19203, 7829.99982, 10000000, 9169.60916, 10000000, 10571.28621, 20904.74224]
ratios1 = [n/h for n,h in zip(naives, heur)]
print(ratios1)
ratios2 = [h/true for h,true in zip(heur, y)]
print(ratios2)
ratios3 = [n/true for n,true in zip(naives, y)]
print(ratios3)

In [ ]:
x = [52, 82, 104, 164, 156, 246, 208, 328, 260, 410, 312, 492, 364, 574, 416, 656, 468, 738, 520, 820, 572, 902]

In [ ]:
import matplotlib.pyplot as plt
import math

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])


#print(ub_ratio)
#plt.scatter(obj_lb_lst[:,0], obj_lb_lst[:,1], label='lower bound', color="g", linewidths=0.1)


# plt.scatter(x, y, s=5, label='opt/lb', color='b')
plt.scatter(x, ratios1, label='Naive/Heur', color='k')
plt.scatter(x, ratios2, label='Heur/Optimal', color='r')
plt.scatter(x, ratios3, label='Naive/Optimal', color='b')
#plt.scatter(x, naives, label='Naives', color='g')
plt.xlabel('Number of tasks')
plt.ylabel('Ratios')
plt.legend()
plt.show()

In [ ]:
import csv
with open("1000genome_naive_heur_3_machines.csv", 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(["task_size", "optimal", "naive", "heuristic", "lowerbound", "upperbound"])
    writer.writerows(zip(x, y, naives, heurs, lbs, ubs))